# Example: Online Stochastic Bandit Algorithms for Portfolio Rebalancing
In this example, we'll propose a stochastic multi-armed bandit approach to portfolio rebalancing where a bandit algorithm selects which assets to include in the portfolio, and an optimal Utility Maximization (UM) approach is used to determine the optimal weights for the selected assets. Thus, we simultaneously address the asset selection and weight allocation problems in portfolio management.

> __Learning Objectives__
> 
> By the end of this example, you should be able to:
> * __Apply multi-armed bandit algorithms__ to explore different portfolio compositions by balancing exploration of uncertain subsets with exploitation of known high-utility combinations.
> * __Formulate and solve utility maximization problems__ using Cobb-Douglas utility functions to determine optimal asset share counts for a selected portfolio, with the understanding that this has an analytical solution for asset selections.
> * __Integrate bandit selection with utility maximization__ to demonstrate how these two approaches work together to simultaneously address asset selection and weight allocation problems in portfolio management.

This material will be presented at the upcoming [INFORMS Annual Meeting 2025](https://meetings.informs.org/wordpress/annual/) in Atlanta, GA on October 26-29, 2025. Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include:__ The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [1]:
include(joinpath(@__DIR__, "Include-informs.jl")); # include the Include.jl file

For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

### Data
We gathered daily open-high-low-close (OHLC) data for each firm in the [S&P 500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2025` until `10-17-2025`, along with data for several exchange-traded funds and volatility products during that time period.

Let's load the `original_dataset::DataFrame` by calling [the `MyTestingMarketDataSet()` function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/data/#VLQuantitativeFinancePackage.MyTestingMarketDataSet).

In [2]:
original_out_of_sample_dataset = MyTestingMarketDataSet() |> x-> x["dataset"] # load the original dataset (testing)

Dict{String, DataFrame} with 482 entries:
  "NI"   => 182×8 DataFrame…
  "EMR"  => 182×8 DataFrame…
  "CTAS" => 182×8 DataFrame…
  "HSIC" => 182×8 DataFrame…
  "KIM"  => 182×8 DataFrame…
  "PLD"  => 182×8 DataFrame…
  "IEX"  => 182×8 DataFrame…
  "BAC"  => 182×8 DataFrame…
  "CBOE" => 182×8 DataFrame…
  "EXR"  => 182×8 DataFrame…
  "NCLH" => 182×8 DataFrame…
  "CVS"  => 182×8 DataFrame…
  "DRI"  => 182×8 DataFrame…
  "DTE"  => 182×8 DataFrame…
  "ZION" => 182×8 DataFrame…
  "AVY"  => 182×8 DataFrame…
  "EW"   => 182×8 DataFrame…
  "EA"   => 182×8 DataFrame…
  "NWSA" => 182×8 DataFrame…
  ⋮      => ⋮

Not all tickers in our dataset have the maximum number of trading days for various reasons, such as acquisition or delisting events. Let's collect only those tickers with the maximum number of trading days.

First, let's compute the number of records for a firm that we know has the maximum value, e.g., `AAPL`, and save that value in the `maximum_number_trading_out_of_sample_days::Int64` variable:

In [3]:
maximum_number_trading_out_of_sample_days = original_out_of_sample_dataset["AAPL"] |> nrow; # maximum number of trading days in our dataset 

Now, let's iterate through our data and collect only tickers with `maximum_number_trading_out_of_sample_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [4]:
dataset = let

    # initialize -
    dataset = Dict{String, DataFrame}();

    # iterate through the dictionary; we can't guarantee a particular order
    for (ticker, data) ∈ original_out_of_sample_dataset  # we get each (K, V) pair!
        if (nrow(data) == maximum_number_trading_out_of_sample_days) # check if ticker has maximum trading days
            dataset[ticker] = data;
        end
    end
    dataset; # return
end;

Let's get a list of the firms in our cleaned dataset and sort them alphabetically. We store the sorted firm ticker symbols in the `list_of_tickers_price_data::Array{String,1}` variable:

In [5]:
list_of_tickers_price_data = keys(dataset) |> collect |> sort;

Finally, let's load the single index model parameters that we computed in the previous example. We'll store this data in the `sim_model_parameters::Dict{String,NamedTuple}` variable. In addition, we return a few other useful variables, such as the historical market growth rate, the mean and variance of the market growth, etc.

In [6]:
sim_model_parameters,Gₘ,Ḡₘ, Varₘ = let

    # initialize -
    path_to_sim_model_parameters = joinpath(_PATH_TO_DATA,"SIMs-SPY-SP500-01-03-14-to-12-31-24.jld2");
    sim_model_parameters = JLD2.load(path_to_sim_model_parameters);
    parameters = sim_model_parameters["data"]; # return

    Gₘ = sim_model_parameters["Gₘ"]; # Get the past market growth rate 
    Ḡₘ = sim_model_parameters["Ḡₘ"]; # mean of market growth rates
    Varₘ = sim_model_parameters["Varₘ"]; # variance of market growth

    # return -
    parameters,  Gₘ , Ḡₘ, Varₘ;
end;

Now let's get a list of all tickers for which we have single index model parameters:

In [7]:
tickers_that_we_sim_sim_data_for = keys(sim_model_parameters) |> collect |> sort;

We need to use only the tickers for which we have both price data and SIM parameters. We'll compute [the intersection of the two lists](https://docs.julialang.org/en/v1/base/collections/#Base.intersect) and store the result in the `list_of_tickers::Array{String,1}` variable:

In [8]:
list_of_tickers = intersect(tickers_that_we_sim_sim_data_for, list_of_tickers_price_data);

Finally, we need to compute the projection matrix `X̄::Array{Float64,2}` = $(\mathbf{X}^{\top}\mathbf{X})^{-1}\mathbf{X}^{\top}$ (which will be handy for later) where $\mathbf{X}$ is the market factor matrix:

In [9]:
X̄ = let
    
    
    Rₘ = Gₘ; # get Gₘ from the sim model parameters
    max_length = length(Rₘ);
    X = [ones(max_length) Rₘ];

    a = inv(transpose(X)*X)*transpose(X)
    a;
end

2×2766 Matrix{Float64}:
  0.000367293  0.000355057  0.0003607   …   0.000382956   0.000370094
 -5.4289e-5    6.10402e-5   7.85216e-6     -0.000201909  -8.06852e-5

### Constants
Finally, let's set some constants we'll use later in this notebook. The comments describe the constants, their units, and permissible values:

In [10]:
Δt = (1/252); # time-step
total_number_of_tickers = length(list_of_tickers); # how many tickers do we have in my dataset?
investment_budget = 10000.0; # initial budget of the agent
risk_free_rate = 0.0418; # risk free rate
μₘ =  Ḡₘ; # expected growth for SPY
B = investment_budget; # TODO: Budget for portfolio. We can change this later if we want
ϵ = 0.001; # hyperparameter: minimum share number for each asset
α::Float64 = 0.80; # learning rate for moving average calculation
Tₘ::Int64 = 10; # how many days for market time
λ::Float64 = 2.0; # risk scale
number_of_days_to_simulate = maximum_number_trading_out_of_sample_days; # how many days to simulate

### Implementation
Next, build the `world(...)` function. The `world(...)` function takes the action vector `a::Vector{Int64}` where the elements of `a::Vector{Int64}` are binary variables indicating whether to select an asset (`1`) or not (`0`). The length of the action vector `a` is $|\dim\mathcal{P}|$, i.e., the total number of assets available for selection.

> __What's in the action vector?__ The action vector `a::Vector{Int64}` is a binary vector of length $|\dim\mathcal{P}|$ where each element indicates whether to include (`1`) or exclude (`0`) the corresponding asset from the portfolio. For example, if we have 5 assets and the action vector is `[1, 0, 1, 0, 1]`, it means we are selecting assets 1, 3, and 5 for inclusion in the portfolio while excluding assets 2 and 4.

The `world(...)` function returns the portfolio Utility, the optimal number of shares for each selected asset, the fill price for each selected asset, and the user preference values $\gamma_i$ for each selected asset. 

In [11]:
function world(t::Int, a::Vector{Int64}, context::MyDynamicBanditPortfolioAllocationContextModel)

    # initialize -
    total_number_of_assets = context.number_of_assets; # total number of assets that we *could* purchase
    bounds = context.bounds; # bounds for how much we purchase
    B = context.B; # budget for this period
    mylocaltickers = context.tickers; # tickers for the assets in this portfolio
    localdataset = context.dataset; # dataset for the assets in this portfolio
    singleindexmodels = context.singleindexmodels; # single index models for the assets in this portfolio
    X̄ = context.X̄;
    number_of_samples_to_draw = context.number_of_samples_to_draw;

    # what is the min share purchase?
    min_share_purchase = bounds[1,1];

    # Compute the fill price vector -
    pₒ = zeros(total_number_of_assets);
    for i ∈ eachindex(mylocaltickers)
        ticker = mylocaltickers[i];
        H = localdataset[ticker][t,:high];
        L = localdataset[ticker][t,:low];
        f = rand();
        pₒ[i] = f*H + (1-f)*L; # randomness in the fill price
    end

    # next, compute the preference parameters γ -
    γ = zeros(total_number_of_assets);
    for i ∈ eachindex(mylocaltickers)
        ticker = mylocaltickers[i];

        # get the model -
        simmodel = singleindexmodels[ticker];
        α = simmodel.alpha;
        β = simmodel.beta;
        CI_alpha_L = simmodel.alpha_95_CI_lower;
        CI_alpha_U = simmodel.alpha_95_CI_upper;
        CI_beta_L = simmodel.beta_95_CI_lower;
        CI_beta_U = simmodel.beta_95_CI_upper;

        # compute random parameters -
        # αᵢ = rand(Uniform(CI_alpha_L, CI_alpha_U)); # sample from the confidence interval
        # βᵢ = rand(Uniform(CI_beta_L, CI_beta_U));
        αᵢ = α; # use the mean value
        βᵢ = β; # use the mean value

        # Compute the alpha and beta values -
        R = αᵢ + βᵢ*μₘ; # draw random value from the error distribution -
        γ[i] = tanh_fast(R/(βᵢ^λ));
    end

    # Compute the optimal share count -
    n = zeros(total_number_of_assets); # initialize space for optimal solution
    S = findall(aᵢ -> aᵢ == 1, a); # Which assets does our bandit want us to buy?
    number_of_selected_arms = length(S);

    # In the set of assets to explore, do we have any non-preferred assets?
    negative_gamma_flag = any(γ[S] .< 0);
    if (negative_gamma_flag == false)
        
        # easy case: all of my potential assets are preferred.
        γ̄ = sum(γ[S]);
        B̄ = B;
        for s ∈ S
            n[s] = (γ[s]/γ̄)*(B̄/pₒ[s]);
        end
    else

        # hard case: some assets are *not* preferred. 
        
        # Prep work for non-preferred case
        # First: the non-preferred assets are min_share_purchase -
        # Second: Compute the adjusted budget
        # Third: Compute γ̄
        B̄ = B;
        γ̄ = 0.0;
        for s ∈ S
            if (γ[s] < 0.0)
                B̄ += -min_share_purchase*pₒ[s];
                n[s] = min_share_purchase;
            else
                γ̄ += γ[s];
            end
        end

        # compute the optimal preferred assets -
        for s ∈ S
            if (γ[s] ≥ 0.0)
                n[s] = (γ[s]/γ̄)*(B̄/pₒ[s]);
            end
        end
    end

    # premultiplier -
    κ = negative_gamma_flag == true ? -1.0 : 1.0;
    
    # Compute the optimal utility -
    U = κ;
    for s ∈ S
        U *= (n[s]^γ[s])
    end
    
    # Return the utility and the share count that we allocated
    return U, n, pₒ, γ
end;

___

## Task 1: Maximum Utility Portfolio Optimization Problem
In this task, we'll solve the maximum utility portfolio optimization problem for a given set that we select such that we have the maximum __investor satisfaction__, i.e., maximum Utility. Let's start by specifying the possible tickers that our online stochastic bandit algorithm can choose from. We'll store these tickers in the `my_test_portfolio_tickers::Array{String,1}` variable:

In [12]:
my_test_portfolio_tickers = ["AAPL", "MSFT", "INTC", "MU", "AMD", "GS", "BAC", "WFC", "C", "F", "GM", 
    "JNJ", "PG", "UPS", "COST", "TGT", "WMT", "MRK", "PFE", "ADBE"]; # tickers selected for portfolio

In [13]:
length(my_test_portfolio_tickers) # how many tickers do we have?

20

Given this set of possible tickers, let's compute the expected growth rate vector and the covariance matrix for these assets using their single index model parameters. We don't use these values in the bandit calculation, but we will need them to compute the Sharpe ratio of the resulting portfolio later. 

We'll store these in the `μ̂_sim::Array{Float64,1}` and `Σ̂_sim::Array{Float64,2}` variables:

In [14]:
μ̂_sim, Σ̂_sim = let

    # initialize -
    N = length(my_test_portfolio_tickers); # number of assets in portfolio
    μ_sim = Array{Float64,1}(); # drift vector
    Σ̂_sim = Array{Float64,2}(undef, N, N); # covariance matrix for *our* portfolio
    
    # Ḡₘ = mean(Gₘ); # mean of market factor # TODO: already loaded above, we'll use 2014 - 2024 data value
    σ²ₘ = Varₘ; # variance of market factor # TODO: already loaded above, we'll use 2014 - 2024 data value

    # compute the expected growth rate (return) for each of our tickers -
    for i ∈ eachindex(my_test_portfolio_tickers)
        ticker = my_test_portfolio_tickers[i];
        data = sim_model_parameters[ticker]; # get the data for this ticker
        αᵢ = data.alpha; # get alpha
        βᵢ = data.beta; # get beta
        Ḡᵢ = αᵢ + βᵢ* Ḡₘ; # compute the growth rate for this ticker
        push!(μ_sim, Ḡᵢ); # append growth rate value to μ_sim
    end

    # compute the covariance matrix using the single index model -
    for i ∈ eachindex(my_test_portfolio_tickers)

        ticker_i = my_test_portfolio_tickers[i];
        data_i = sim_model_parameters[ticker_i]; # get the data for ticker i
        βᵢ = data_i.beta; # get beta for ticker i
        σ²_εᵢ = (Δt)*data_i.training_variance; # residual variance for ticker i

        for j ∈ eachindex(my_test_portfolio_tickers)
            
            ticker_j = my_test_portfolio_tickers[j];
            data_j = sim_model_parameters[ticker_j]; # get the data for ticker j
            βⱼ = data_j.beta; # get beta for ticker j
            σ²_εⱼ = (Δt)*data_j.training_variance; # residual variance for ticker j
            
            if i == j
                Σ̂_sim[i,j] = βᵢ*βⱼ*σ²ₘ + σ²_εᵢ; # diagonal elements
            else
                Σ̂_sim[i,j] = βᵢ*βⱼ*σ²ₘ; # off-diagonal elements
            end
        end
    end

    (μ_sim, Σ̂_sim*Δt); # return
end;

To implement our bandit strategy, we first build the epsilon-greedy dynamic noise algorithm model. This model holds some information about the problem, e.g., the number of assets available for selection, and the learning rate $\alpha$. We store this model in the `dynamic_algorithm_model::MyEpsilonGreedyDynamicNoiseAlgorithmModel` variable:

In [15]:
dynamic_algorithm_model = let

    # initialize -
    algorithm = nothing; # Initialize the algorithm variable to nothing; this variable will be used to store the algorithm model
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    
    # TODO: Build an algorithm model by uncommenting the code block below
    algorithm = build(MyEpsilonGreedyDynamicNoiseAlgorithmModel, (
        K = K, # arms 
        α = α, # learning rate
    ));

    # return the algorithm -
    algorithm;
end;

Next, we set bounds on the number of shares that we can hold for each asset in our portfolio. The lower bound `ϵ::Float64` represents the __minimum number of shares__ required for portfolio inclusion, while the upper bound is unbounded to allow for flexible allocation. 

> __Why a non-zero lower bound?__ Setting a non-zero lower bound `ϵ::Float64` follows from the form of the Cobb-Douglas utility function, if any asset has zero allocation, the overall utility becomes zero which does not reflect realistic investor behavior. Thus, we set a small positive value for `ϵ::Float64` to ensure that each selected asset contributes positively to the portfolio's utility.

We'll store this in the `static_share_bounds::Array{Float64,2}` variable:

In [16]:
static_share_bounds = let

    # initialize -
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    bounds = Array{Float64,2}(undef, K, 2);
    
    # build my bounds array -
    for i ∈ eachindex(my_test_portfolio_tickers)
        bounds[i,1] = ϵ; # min shares that we can hold of this asset
        bounds[i,2] = Inf; # max number of shares, Inf says this is unbounded
    end
    bounds;
end;

Now we build the context model that encapsulates all the data and parameters needed for the bandit algorithm to make portfolio decisions. The context includes the market data, single index model parameters, ticker symbols, budget constraints, and bounds on share purchases. We'll store this in the `dynamic_context_model::MyDynamicBanditPortfolioAllocationContextModel` variable:

In [17]:
dynamic_context_model = let

    # initialize -
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    nₒ = ones(K); # initial guess, assume 1 x share for each

    # build -
    contextmodel = build(MyDynamicBanditPortfolioAllocationContextModel, (
        dataset = dataset,
        singleindexmodels = sim_model_parameters,
        tickers = my_test_portfolio_tickers,
        nₒ = nₒ,
        B = B,
        bounds = static_share_bounds,
        X̄ = X̄,
        R̄ₘ = Ḡₘ,
        number_of_samples_to_draw = size(X̄,2),
        μₒ = zeros(2^K) # initially we have *no* idea which arm is best
    ));

    contextmodel;
end;

Finally, we test our approach by solving the utility maximization problem for the case where we select all available assets (by setting the action vector `a::Vector{Int64}` to all ones). This gives us a baseline utility `U::Float64` we could achieve with our full portfolio. 

> __Expectation:__ If this doesn't blow up, then it will return a finite utility value, optimal share counts, fill prices, and preference parameters for each asset in the portfolio. Then we can move on to implementing the bandit algorithm to select subsets of assets dynamically.

So what do we see?

In [18]:
(U, n, pₒ, γ) = let
   
    # call my world function to test -
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    t = 1; # what time period are we in?

    # call the world function directly -
    U, n, pₒ, γ = world(t, ones(Int64,K), dynamic_context_model);
end;

In [19]:
U

-45.708788445333965

__Preferred versus Non-Preferred Assets:__ The `γ::Array{Float64,1}` vector computed above indicates whether an asset is preferred or non-preferred. If $\gamma_i \geq 0$, then the asset is preferred; otherwise, it is non-preferred. Let's look at what assets are preferred versus non-preferred:

In [20]:
let

    # initialize -
    df = DataFrame();

    for i ∈ eachindex(my_test_portfolio_tickers)
        ticker = my_test_portfolio_tickers[i];

        # get the model -
        row_df = (
            ticker = ticker,
            γᵢ = γ[i],
            preferred = γ[i] >= 0 ? "Yes" : "No",

        );
        push!(df, row_df);
    end

    # make a table -
     pretty_table(df, backend = :text, fit_table_in_display_vertically = false, fit_table_in_display_horizontally = false,
         table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end

 -------- ------------ -----------
  ticker           γᵢ   preferred 
  String      Float64      String 
 -------- ------------ -----------
    AAPL     0.161704         Yes
    MSFT     0.165848         Yes
    INTC   -0.0162433          No
      MU    0.0440374         Yes
     AMD     0.102785         Yes
      GS    0.0619439         Yes
     BAC    0.0485109         Yes
     WFC    0.0260658         Yes
       C    0.0114522         Yes
       F   -0.0197825          No
      GM    0.0124328         Yes
     JNJ     0.140039         Yes
      PG     0.270409         Yes
     UPS    0.0234057         Yes
    COST     0.350777         Yes
     TGT    0.0996727         Yes
     WMT     0.453629         Yes
     MRK     0.198229         Yes
     PFE   -0.0337729          No
    ADBE      0.11439         Yes
 -------- ------------ -----------


Now that we have the preferred versus non-preferred assets, we'll make a table that summarizes the resulting shares `n::Array{Float64,1}` purchased for each asset in our test portfolio:

In [21]:
let

    # initialize -
    df = DataFrame();
    CS = 0;
    t = 1;
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    context = dynamic_context_model;
    total_number_of_assets = context.number_of_assets; # total number of assets that we *could* purchase
    bounds = context.bounds; # bounds for how much we purchase
    B = context.B; # budget for this period
    mylocaltickers = context.tickers; # tickers for the assets in this portfolio
    localdataset = context.dataset; # dataset for the assets in this portfolio
    singleindexmodels = context.singleindexmodels; # single index models for the assets in this portfolio
    N = 2^K; # number of possible portfolios
    

    for i ∈ 1:K

        CS += n[i]*pₒ[i];
        wᵢ = (n[i]*pₒ[i]/B);

        row_df = (
            ticker = my_test_portfolio_tickers[i],
            Sᵢ = pₒ[i],
            γᵢ = γ[i],
            nᵢ = n[i],
            wᵢ = wᵢ,
            UC = n[i]*pₒ[i],
            CS = CS,
        );
        push!(df, row_df);
    end

    pretty_table(df, backend = :text, fit_table_in_display_vertically = false, fit_table_in_display_horizontally = false,
         table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end

 -------- --------- ------------ ---------- ------------ ------------ ---------
  ticker        Sᵢ           γᵢ         nᵢ           wᵢ           UC        CS 
  String   Float64      Float64    Float64      Float64      Float64   Float64 
 -------- --------- ------------ ---------- ------------ ------------ ---------
    AAPL   241.908     0.161704    2.92496     0.070757       707.57    707.57
    MSFT   421.248     0.165848    1.72274    0.0725702      725.702   1433.27
    INTC   20.3577   -0.0162433      0.001   2.03577e-6    0.0203577   1433.29
      MU   89.5117    0.0440374    2.15274    0.0192695      192.695   1625.99
     AMD   123.972     0.102785     3.6279    0.0449758      449.758   2075.75
      GS   573.739    0.0619439   0.472424    0.0271048      271.048   2346.79
     BAC   44.6785    0.0485109    4.75105    0.0212269      212.269   2559.06
     WFC   70.3038    0.0260658    1.62233    0.0114056      114.056   2673.12
       C   70.4051    0.0114522   0.711759   0.0

## Task 2: Solve the Online Bandit Portfolio Optimization Problem
In this task, we use the epsilon-greedy bandit algorithm to explore different portfolio combinations and learn which asset subsets maximize investor utility. 

The algorithm iteratively selects actions (portfolio compositions), observes the resulting utility, and updates its beliefs about the reward distribution. The results are stored in the return variables `R::Array{Float64,2}`, `μ::Array{Float64,1}`, `S::Array{Float64,2}`, `P::Array{Float64,2}`, and `G::Array{Float64,2}`. 

Let's run the algorithm for 250 iterations:

In [22]:
R, μ, S, P, G = my_bandit_solve(dynamic_algorithm_model, T = 500, world = world, context = dynamic_context_model); # check the world function

In [23]:
μ

1048576-element Vector{Float64}:
  0.0036473818858245716
  0.0033801760709609593
  0.004917507923109867
 -0.0022374836177335806
 -0.004078732984973006
 -0.003780281276862472
 -0.005497214998317741
  0.002462481187007658
  0.004036161981649106
  0.003732988481199112
  ⋮
 -0.08555005636566723
 -0.08761924350857282
 -0.080590933116772
 -0.08164102711575313
 -0.09497736274338066
 -0.09796791536769606
 -0.0897360883785586
 -0.09144908045228045
 -0.09166065204237431

In [24]:
S # share counts over for each trial

500×20 Matrix{Float64}:
  2.90302  1.71396  0.001  2.18381  …  22.0202   8.71213  0.001  1.17118
  8.32132  0.0      0.001  0.0          0.0      0.0      0.0    3.29041
 10.7823   0.0      0.0    8.01336      0.0      0.0      0.0    0.0
  0.0      0.0      0.001  0.0         43.1642   0.0      0.001  2.28683
  0.0      0.0      0.001  5.66387      0.0     23.0191   0.0    3.1102
  0.0      0.0      0.0    0.0      …   0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0          0.0     31.5125   0.001  0.0
  4.61188  2.72702  0.001  0.0         34.6448   0.0      0.0    0.0
  4.11467  0.0      0.001  0.0         30.8852  12.3363   0.001  1.64853
  0.0      0.0      0.0    0.0         44.1677  17.5992   0.001  0.0
  ⋮                                 ⋱                            
  0.0      0.0      0.0    0.0         52.264   20.9609   0.0    0.0
  7.72492  4.56679  0.0    5.83907      0.0      0.0      0.0    3.08987
  0.0      0.0      0.0    0.0         52.2898  20.8116   0

We can sort the possible portfolio compositions by the estimated average utility `μ::Array{Float64,1}` to see which portfolios the bandit algorithm thinks are best:

In [25]:
sorted_reward_index_array = sortperm(μ, rev=true) # sort the index of the rewards

1048576-element Vector{Int64}:
  199760
  721400
  223643
  599225
  695402
  553049
  639306
  625897
  151683
  564664
       ⋮
  654670
  881822
  367432
  523585
  989930
  497413
 1028954
  481854
  204567

__What's in the best portfolio?__  
Let's build a table that shows the optimal portfolio composition identified by the bandit algorithm after 250 periods of exploration. 

The utility value `U::Float64` indicates the investor satisfaction level for this portfolio, while the weights `wᵢ::Float64` show the proportion of budget allocated to each asset. Notice that the bandit algorithm may have selected a portfolio subset (where some `nᵢ::Float64` values are near zero) rather than including all assets, demonstrating how the algorithm balances portfolio complexity with investor utility.

In [26]:
let

    # initialize -
    df = DataFrame();
    CS = 0;
    t = 1;
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    context = dynamic_context_model;
    total_number_of_assets = context.number_of_assets; # total number of assets that we *could* purchase
    bounds = context.bounds; # bounds for how much we purchase
    B = context.B; # budget for this period
    mylocaltickers = context.tickers; # tickers for the assets in this portfolio
    localdataset = context.dataset; # dataset for the assets in this portfolio
    singleindexmodels = context.singleindexmodels; # single index models for the assets in this portfolio
    N = 2^K; # number of possible portfolios

    # compute the best portfolio -
    j = sorted_reward_index_array[1]; # get a portfolio index 
    a = digits(j, base=2, pad=K); # generate a binary representation of the number, with K digits  
    if (j == N)
        a = digits(j - 1, base=2, pad=K); # generate a binary representation of the number, with K digits  
    end

    # call world -
    U, n, pₒ, γ = world(t, a, dynamic_context_model);

    @show U,j

    for i ∈ 1:K

        CS += n[i]*pₒ[i];
        wᵢ = (n[i]*pₒ[i]/B);

        row_df = (
            ticker = my_test_portfolio_tickers[i],
            Sᵢ = pₒ[i],
            γᵢ = γ[i],
            nᵢ = n[i],
            wᵢ = wᵢ,
            UC = n[i]*pₒ[i],
            CS = CS,
        );
        push!(df, row_df);
    end

     pretty_table(df, backend = :text, fit_table_in_display_vertically = false, fit_table_in_display_horizontally = false,
         table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end

(U, j) = (21.624045526899064, 199760)
 -------- --------- ------------ --------- ----------- --------- ---------
  ticker        Sᵢ           γᵢ        nᵢ          wᵢ        UC        CS 
  String   Float64      Float64   Float64     Float64   Float64   Float64 
 -------- --------- ------------ --------- ----------- --------- ---------
    AAPL   244.015     0.161704       0.0         0.0       0.0       0.0
    MSFT   423.508     0.165848       0.0         0.0       0.0       0.0
    INTC   20.1252   -0.0162433       0.0         0.0       0.0       0.0
      MU    89.691    0.0440374       0.0         0.0       0.0       0.0
     AMD   122.903     0.102785   8.75147    0.107558   1075.58   1075.58
      GS   574.902    0.0619439       0.0         0.0       0.0   1075.58
     BAC   44.2688    0.0485109   11.4671   0.0507634   507.634   1583.21
     WFC   70.5269    0.0260658       0.0         0.0       0.0   1583.21
       C   70.7297    0.0114522       0.0         0.0       0.0   1583

Ok, so what is the Sharpe ratio of this portfolio?

In [27]:
let

    # initialize -
    t = 1;
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    N = 2^K; # number of possible portfolios

    # compute the best portfolio -
    j = sorted_reward_index_array[1]; # get a portfolio index 
    a = digits(j, base=2, pad=K); # generate a binary representation of the number, with K digits  
    if (j == N)
        a = digits(j - 1, base=2, pad=K); # generate a binary representation of the number, with K digits  
    end

    # call world -
    U, n, pₒ, γ = world(t, a, dynamic_context_model);

    # compute the weights -
    w = zeros(K);
    for i ∈ 1:K
        w[i] = (n[i]*pₒ[i])/B;
    end

    # get the alpha and beta values for each asset in the portfolio -
    α_vector = zeros(K);
    β_vector = zeros(K);

    # compute the alpha for the portfolio -
   for i ∈ 1:K
       ticker = my_test_portfolio_tickers[i];
       α_vector[i] = sim_model_parameters[ticker].alpha;
       β_vector[i] = sim_model_parameters[ticker].beta;
   end

   # compute the alpha and beta for the portfolio -
   α_portfolio = dot(w, α_vector);
   β_portfolio = dot(w, β_vector);
   numerator = α_portfolio + β_portfolio*μₘ - risk_free_rate;
   denominator = sqrt(transpose(w)*Σ̂_sim*w);
   sharpe_ratio = numerator/denominator;

   println("The Sharpe ratio of this portfolio is: $(sharpe_ratio)");
end

The Sharpe ratio of this portfolio is: 0.5013335454240974


## Task 3: Compute Maximium Sharpe Ratio Portfolio
In this task, we'll compute the maximum Sharpe ratio portfolio using the expected returns and covariance matrix derived from the single index model parameters. This will serve as a benchmark to compare against the portfolios generated by our bandit algorithm.

To solve the maximum Sharpe ratio problem, we reformulate it as a Second Order Cone Program (SOCP). We need to specify a lower bound for an auxiliary variable $\tau$ (where $\text{SR}(w)\ge \tau$), and build an optimization problem model.

We'll compute the maximum Sharpe ratio portfolio by constructing an optimization problem. We need to build a `MySharpeRatioPortfolioChoiceProblem` object to hold the parameters and bounds for the portfolio optimization problem. We'll store this in the `model::MySharpeRatioPortfolioChoiceProblem` variable:

In [28]:
model = let

    # initialize - 
    my_list_of_tickers = my_test_portfolio_tickers;
    N = length(my_list_of_tickers); # number of assets in portfolio
    α = zeros(N);
    β = zeros(N);

    # grab alphas and betas for our tickers -
    for i ∈ eachindex(my_list_of_tickers)
        ticker = my_list_of_tickers[i];
        data = sim_model_parameters[ticker]; # get the data for this ticker
        α[i] = data.alpha; # get alpha
        β[i] = data.beta; # get beta
    end

    # build the model
    model = VLQuantitativeFinancePackage.build(MySharpeRatioPortfolioChoiceProblem, (
        Σ = Σ̂_sim,
        risk_free_rate = risk_free_rate,
        gₘ = Ḡₘ,
        α = α,
        β = β,
        τ = 1.0, # placeholder, will be set later
    ));

    model; # return -
end;

 We compute the (initial value) of the lower bound $L$ by finding the maximum ratio of excess growth to standard deviation for __each individual asset__:
$$
L = \max_i \left\{\frac{\alpha_i + \beta_i \mathbb{E}[g_M] - r_f}{\sqrt{\Sigma_{ii}}} \right\}\quad\forall{i}\in\mathcal{P}
$$
where $\alpha_i$ is the asset's alpha, $\beta_i$ is the asset's beta, $\mathbb{E}[g_M]$ is the expected market return, $r_f$ is the risk-free rate, and $\Sigma_{ii}$ is the asset's variance.
This bound is stored in the `L::Float64` variable. The parameter $\tau$ must be at least as large as this lower bound for the SOCP to have an optimal solution with a well-defined Sharpe ratio.

In [29]:
L = let

    # initialize -
    α = model.α
    β = model.β
    rfr = model.risk_free_rate
    gₘ = model.gₘ
    Σ = model.Σ
    d = length(α)
    c = α .+ β .* gₘ .- rfr .* ones(d); # vector c

    tau_lower_bound_array = Array{Float64,1}();
    for i ∈ 1:d
        value = c[i]/sqrt(Σ[i,i]);
        push!(tau_lower_bound_array, value);
    end

    L = max(0, maximum(tau_lower_bound_array) ); # return
end

0.8369487745462407

Now we solve the maximum Sharpe ratio portfolio problem using the [solve(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/portfolio/#VLQuantitativeFinancePackage.solve-Tuple{MySharpeRatioPortfolioChoiceProblem}). The solution is stored in the `sharpe_solution::NamedTuple` variable, which contains the risk value, reward value, allocation weights, and the Sharpe ratio for the optimal portfolio. 

> __Solver note:__ The [solve(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/portfolio/#VLQuantitativeFinancePackage.solve-Tuple{MySharpeRatioPortfolioChoiceProblem}) uses the [JuMP.jl package](https://jump.dev/JuMP.jl/stable/) to formulate and solve the optimization problem. JuMP provides a high-level interface for defining optimization problems and supports various solvers. In this case, we use the [`COSMO.jl` solver from the University of Oxford](https://oxfordcontrol.github.io/COSMO.jl/stable/). 

In the code blow below, we compute the optimal Sharpe ratio portfolio by iteratively increasing the lower bound $L$ until until we no longer have an optimal solution, i.e., until our problem is no longer feasible.

In [30]:
sharpe_solution = let
   
    # initialize -
    solution = nothing;
    Lᵢ = L; # store initial lower bound value
    should_stop_loop = false; # we'll keep iterating until we are no longer optimal

    while should_stop_loop == false
         
        # solve the problem with τ = L (the computed lower bound)
        model.τ = Lᵢ; # set the lower bound on the desired Sharpe ratio for this iteration
        solution_dictionary = VLQuantitativeFinancePackage.solve(model) # solve for this Lᵢ
        status_flag = solution_dictionary["status"]; # get the status flag
        
        if (status_flag == MathOptInterface.OPTIMAL) # if optimal solution found, grab the results
            risk_value = solution_dictionary["denominator"]; # risk value (denominator of Sharpe ratio)
            reward_value = solution_dictionary["numerator"]; # reward value (numerator of Sharpe ratio)
            allocation = solution_dictionary["argmax"]; # allocation weights
            sharpe_ratio = solution_dictionary["sharpe_ratio"]; # computed Sharpe ratio
        
            solution = (risk = risk_value, reward = reward_value, w = allocation, 
                sharpe_ratio = sharpe_ratio, status = status_flag);

            # update the Sharpe ratio lower bound -
            Lᵢ += 0.001; # increment lower bound and go around again
        else
            should_stop_loop = true; # Bam! We blew up. Exit the loop.
        end
    end

    solution; # return
end;

------------------------------------------------------------------
          COSMO v0.8.9 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2022
------------------------------------------------------------------

Problem:  x ∈ R^{20},
          constraints: A ∈ R^{43x20} (290 nnz),
          matrix size to factor: 63x63,
          Floating-point precision: Float64
Sets:     Nonnegatives of dim: 21
          SecondOrderCone of dim: 21
          ZeroSet of dim: 1
Settings: ϵ_abs = 1.0e-05, ϵ_rel = 1.0e-05,
          ϵ_prim_inf = 1.0e-04, ϵ_dual_inf = 1.0e-04,
          ρ = 0.1, σ = 1e-06, α = 1.6,
          max_iter = 50000,
          scaling iter = 10 (on),
          check termination every 25 iter,
          check infeasibility every 40 iter,
          KKT system solver: COSMO.QdldlKKTSolver
Acc:      Anderson Type2{QRDecomp},
          Memory size = 15, RestartedMemory,	
          Safeguarded: true, tol: 2.0
Setu

In [31]:
sharpe_solution

(risk = 0.16236078767891132, reward = 0.17192407616922736, w = [0.25333508825254153, 0.2875880574554104, -1.622086428347008e-5, -9.389968640708614e-6, 0.05989215041431877, -7.32470853677147e-6, -9.096040064505045e-6, -1.1833210603662063e-5, -1.5417989206653268e-5, -1.997033929596048e-5, -1.5042185633232818e-5, -5.151583141939685e-6, -2.6850792301166074e-6, -1.0088207425621601e-5, 0.36524872331581043, -5.7274036623780346e-6, 0.0340785990119089, -3.6366325452087172e-6, -1.0160926743210856e-5, -8.73313413283553e-7], sharpe_ratio = 1.0589014664626328, status = OPTIMAL)

__What's in the optimal Sharpe ratio portfolio?__ Let's take a look at the makeup of that portfolio. First, we'll compute the number of shares that we need to purchase (given our investment budget) for each asset. We'll store this in the `nₒ::Array{Float64,1}` variable:

In [32]:
n_sharpe = let
    
    # initialize -
    my_list_of_tickers = my_test_portfolio_tickers;
    N = length(my_list_of_tickers); # number of assets in portfolio
    n = zeros(Float64, N); # number of shares to purchase for each asset
    w = sharpe_solution.w .|> x -> round(x, digits=4) |> abs # optimal weights
    prices = Array{Float64,1}(undef, N); # prices vector

    # get prices for each asset
    for i ∈ eachindex(my_list_of_tickers)
        ticker = my_list_of_tickers[i];
        prices[i] = dataset[ticker][1, :open]; # get the opening price on Jan 3, 2025
    end

    # get the number of shares to purchase for each asset
    for i in 1:N
        n[i] = investment_budget * w[i] / prices[i];
    end

    n; # return
end;

Let's build a table to summarize the maximum Sharpe ratio portfolio allocation. The table shows the ticker, alpha, beta, allocation weight, current price, number of shares, and total cost for each asset in the portfolio:

In [33]:
let

    # initialize -
    df = DataFrame();
    nₒ = n_sharpe; # number of shares to purchase for each asset
    my_list_of_tickers = my_test_portfolio_tickers;

    for i ∈ eachindex(my_list_of_tickers)
        ticker = my_list_of_tickers[i];
        α = sim_model_parameters[ticker].alpha;
        β = sim_model_parameters[ticker].beta;
        price = dataset[ticker][1, :open]; # get the opening price on Jan 3, 2025
        weight = sharpe_solution.w[i] |> x -> round(x, digits=4) |> abs;
        shares = nₒ[i] |> x -> round(x, digits=4);
       
        row_df = (
            ticker = ticker,
            α = round(α, digits=4),
            β = round(β, digits=4),
            w = weight,
            price = round(price, digits=4),
            n = shares,
            cost = round(shares * price, digits=2),
        )
        push!(df, row_df);
    end

    beta_vec = df[:,:β] |> collect;
    alpha_vec = df[:,:α] |> collect;
    w_vec = df[:,:w] |> collect;
    price_vec = df[:,:price] |> collect;

    # compute the total -
    total = df[:,:w] |> sum
    last_row = (
        ticker = "total",
        α = dot(alpha_vec,w_vec),
        β = dot(beta_vec,w_vec),
        w = total,
        price = 0.0,
        n = sum(nₒ),
        cost = round(sum(df[:,:cost]), digits=2),
    )
    push!(df,last_row)

    pretty_table(df, backend = :text, fit_table_in_display_vertically = false,
         table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end

 -------- ---------- --------- --------- --------- --------- ---------
  ticker          α         β         w     price         n      cost 
  String    Float64   Float64   Float64   Float64   Float64   Float64 
 -------- ---------- --------- --------- --------- --------- ---------
    AAPL     0.1061    1.1946    0.2533    243.36   10.4084   2532.99
    MSFT     0.0999     1.152    0.2876    421.08    6.8301   2876.02
    INTC    -0.1484    1.1843       0.0     20.39       0.0       0.0
      MU    -0.0532    1.6953       0.0     87.95       0.0       0.0
     AMD     0.1275    1.7392    0.0599    121.65     4.924     599.0
      GS    -0.0326      1.31       0.0     581.0       0.0       0.0
     BAC    -0.0545    1.3594       0.0     44.75       0.0       0.0
     WFC    -0.0914    1.2383       0.0     70.35       0.0       0.0
       C    -0.1334    1.5008       0.0     70.88       0.0       0.0
       F    -0.1928    1.4336       0.0      9.69       0.0       0.0
      GM    -0.1

Now, let's compute the performance of investing all our budget in `SPY.`

In [34]:
SPY_performance_array = let

    # initialize -
    SPY_performance_array = Array{Float64,1}(undef, number_of_days_to_simulate);
    total_budget = investment_budget; # total budget to invest in SPY
    SPY_df = dataset["SPY"];
    
    # how many shares of SPY do we buy?
    Sₒ = SPY_df[1, :open]; # first open price
    nₒ = total_budget/Sₒ;

    # compute the performance
    for i ∈ 1:number_of_days_to_simulate
        S = SPY_df[i, :close];
        SPY_performance_array[i] = S*nₒ;
    end
    SPY_performance_array;
end;

Next, let's compute the performance of portfolio $\mathcal{P}$ by computing the value of each asset in the portfolio and the total value of the portfolio, starting from the initial allocation. We'll store this data in the `portfolio_performance_array::Array{Float64,2}` array.

Each `row` of the `portfolio_performance_array` corresponds to a `date`, while each `column` corresponds to an asset in the portfolio, with the last column holding the total value of the portfolio $\mathcal{P}$ at a particular time. We'll use the close price of each asset.

In [35]:
portfolio_performance_array = let
    
    number_of_days = number_of_days_to_simulate;
    startdate = Date("2025-01-03"); # starting date for portfolio performance tracking
    my_list_of_tickers = my_test_portfolio_tickers;
    nₒ = n_sharpe; # number of shares to purchase for each asset

    w = sharpe_solution.w .|> x -> round(x, digits=4) |> abs # optimal weights
    portfolio_performance_array = Array{Float64,2}(undef, number_of_days, length(w)+1)
    for i ∈ eachindex(my_list_of_tickers)
        
        ticker = my_list_of_tickers[i];
        price_df = dataset[ticker];
        ticker_data = filter(:timestamp => x-> x >= startdate, price_df)
        nᵢ = nₒ[i]
        
        for j ∈ 1:number_of_days
            portfolio_performance_array[j,i] = nᵢ*ticker_data[j,:close]; # value of this asset in the portfolio at close
        end
    end
    
    # total -
    for i ∈ 1:number_of_days
        portfolio_performance_array[i,end] = sum(portfolio_performance_array[i,1:end-1])
    end
    portfolio_performance_array;
end

182×21 Matrix{Float64}:
 2533.0   2891.5   0.0  0.0  617.317  …  343.383  0.0  0.0  0.0  10046.2
 2550.07  2922.24  0.0  0.0  637.899     345.842  0.0  0.0  0.0  10141.5
 2521.03  2884.81  0.0  0.0  626.968     343.497  0.0  0.0  0.0  10056.4
 2526.13  2899.77  0.0  0.0  599.936     347.241  0.0  0.0  0.0  10077.2
 2465.24  2861.45  0.0  0.0  571.377     351.78   0.0  0.0  0.0   9992.16
 2439.74  2849.43  0.0  0.0  577.679  …  346.22   0.0  0.0  0.0   9906.49
 2428.08  2839.05  0.0  0.0  571.623     343.421  0.0  0.0  0.0   9845.76
 2475.86  2911.72  0.0  0.0  590.679     345.501  0.0  0.0  0.0  10012.4
 2375.83  2899.91  0.0  0.0  583.194     345.35   0.0  0.0  0.0   9877.93
 2393.73  2930.3   0.0  0.0  598.064     347.771  0.0  0.0  0.0  10037.1
    ⋮                                 ⋱                              ⋮
 2463.68  3519.94  0.0  0.0  793.546     392.216  0.0  0.0  0.0  11004.2
 2478.77  3476.77  0.0  0.0  790.099     391.195  0.0  0.0  0.0  10939.6
 2487.52  3483.47  0.0  0

Finally, let's compute the performance of the bandit portfolio.

In [36]:
bandit_portfolio_performance_array = let

    # initialize -
    t = 1;
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    N = 2^K; # number of possible portfolios
    number_of_days = number_of_days_to_simulate;
    my_list_of_tickers = my_test_portfolio_tickers;
    startdate = Date("2025-01-03"); # starting date for portfolio performance tracking

    # compute the best portfolio -
    j = sorted_reward_index_array[1]; # get a portfolio index 
    a = digits(j, base=2, pad=K); # generate a binary representation of the number, with K digits  
    if (j == N)
        a = digits(j - 1, base=2, pad=K); # generate a binary representation of the number, with K digits  
    end

    # call world -
    U, n, pₒ, γ = world(t, a, dynamic_context_model);

    # compute the weights -
    w = zeros(K);
    for i ∈ 1:K
        w[i] = (n[i]*pₒ[i])/B;
    end

    portfolio_performance_array = Array{Float64,2}(undef, number_of_days, length(w)+1)
    for i ∈ eachindex(my_list_of_tickers)
        
        ticker = my_list_of_tickers[i];
        price_df = dataset[ticker];
        ticker_data = filter(:timestamp => x-> x >= startdate, price_df)
        nᵢ = n[i]
        
        for j ∈ 1:number_of_days
            portfolio_performance_array[j,i] = nᵢ*ticker_data[j,:close]; # value of this asset in the portfolio at close
        end
    end
    
    # total -
    for i ∈ 1:number_of_days
        portfolio_performance_array[i,end] = sum(portfolio_performance_array[i,1:end-1])
    end
    portfolio_performance_array;
end

182×21 Matrix{Float64}:
 0.0  0.0  0.0  0.0  1090.5   0.0  514.725  …  2074.51  0.0  0.0  10031.7
 0.0  0.0  0.0  0.0  1126.86  0.0  521.502     2086.64  0.0  0.0  10120.2
 0.0  0.0  0.0  0.0  1107.55  0.0  529.313     2113.85  0.0  0.0  10125.5
 0.0  0.0  0.0  0.0  1059.79  0.0  530.807     2089.36  0.0  0.0  10064.0
 0.0  0.0  0.0  0.0  1009.34  0.0  518.171     2076.81  0.0  0.0  10046.1
 0.0  0.0  0.0  0.0  1020.48  0.0  517.597  …  2113.01  0.0  0.0  10040.5
 0.0  0.0  0.0  0.0  1009.78  0.0  525.867     2085.18  0.0  0.0   9976.91
 0.0  0.0  0.0  0.0  1043.44  0.0  541.03      2094.39  0.0  0.0  10067.9
 0.0  0.0  0.0  0.0  1030.22  0.0  535.746     2107.15  0.0  0.0  10088.9
 0.0  0.0  0.0  0.0  1056.49  0.0  534.482     2048.98  0.0  0.0  10079.5
 ⋮                            ⋮             ⋱                         ⋮
 0.0  0.0  0.0  0.0  1401.81  0.0  581.119     1695.35  0.0  0.0  11062.0
 0.0  0.0  0.0  0.0  1395.72  0.0  581.923     1696.81  0.0  0.0  11033.2
 0.0  0.0  0.0 

### Visualize
`Unhide` the code block below to see how we visualized the relative wealth of the portfolio $W_{t}/W_{o}$ for our selected optimal portfolio versus an alternative portfolio consisting of `SPY` over the same time frame (where the daily price is taken to be the close price).

In [37]:
let
    
    total_budget = investment_budget;
    plot((1/total_budget).*portfolio_performance_array[:,end], lw=3, 
        c=:red, label="Max Sharpe (actual)")
    plot!((1/total_budget).*SPY_performance_array, lw=3, 
        c=:blue, label="SPY")

    plot!((1/total_budget).*bandit_portfolio_performance_array[:,end], lw=3, 
        c=:green, label="Bandit Optimal (actual)")
    xlabel!("Trading Day Index (2025)", fontsize=18)
    ylabel!("Scaled Wealth (N/A)", fontsize=18)

    plot!(bg="gray95", background_color_outside="white", framestyle = :box, fg_legend = :transparent);

    # uncomment me to save the figure -
    path_to_save_figure = joinpath(_PATH_TO_FIGS, "Fig-Wealth-K20-L2-SR-I500.pdf");
    savefig(path_to_save_figure);
end

"/Users/jeffreyvarner/Desktop/julia_work/CHEME-5660-Fall-2025/INFORMS-Annual-Meeting-2025-BanditPoster/figs/Fig-Wealth-K20-L2-SR-I500.pdf"

## Summary
This example demonstrates how online stochastic bandit algorithms can be effectively applied to solve the combined asset selection and weight allocation problems in portfolio management through the integration of epsilon-greedy exploration with utility maximization.

> __Key Takeaways:__
> 
> * __Bandit Algorithms for Portfolio Selection:__ Multi-armed bandit algorithms like epsilon-greedy enable adaptive asset selection by balancing exploration of uncertain portfolios with exploitation of known high-utility combinations, avoiding the need to exhaustively evaluate all possible subsets.
> * __Utility Maximization for Weight Allocation:__ Once a portfolio subset is selected, Cobb-Douglas utility functions provide a principled approach to weight allocation that reflects investor preferences and risk tolerance, with the preference parameters `γ::Array{Float64,1}` distinguishing between preferred and non-preferred assets.
> * __Integration of Selection and Allocation:__ The combination of bandit algorithms for asset selection with Cobb-Douglas utility maximization for weight allocation provides a theoretically sound and computationally efficient framework for simultaneously addressing both problems in portfolio optimization.

By combining these techniques, investors can manage both the complexity of asset selection and the precision of weight allocation in a computationally efficient and theoretically sound framework.
___

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. You should decide solely based on your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.